CHẮC CÓ KIỂM TRA

Nét chữ
Kiểu chữ
Góc chụp

Train       : Huấn luyện
Test        : Kiểm thử
Location

MNIST
tập dữ liệu: Áo quần, chữ số => Tùy nhu cầu

In [3]:
#from google.colab import drive
#drive.mount('/content/gdrive')

SyntaxError: invalid syntax (1123078586.py, line 1)

In [9]:
pip install opencv-python keras tensorflow

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 266.3/266.3 MB 8.2 MB/s eta 0:00:00
     ------------------------------------- 895.9/895.9 kB 27.7 MB/s eta 0:00:00
     ---------------------------------------- 6.0/6.0 MB 23.9 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB 3.5 MB/s eta 0:00:00
     --------------------------------------- 23.2/23.2 MB 24.2 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 23.8 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 26.5 MB/s eta 0:00:00
     ---------------------------------------- 126.5/126.5 kB ? eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB ? eta 0:00:00
     ------------------------------------- 439.2/439.2 kB 28.6 MB/s eta 0:00:00
     ------------------------------------- 781.3/781.3 kB 24.9 MB/s eta 0:00:00
     ---------------------------------------- 177.8/177.8

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Xây dựng model nhận diện, dữ liệu được xây dựng training model là mnist
Đầu tiên load dataset và load các thư viện cần dùng:

In [10]:
import cv2
import numpy as np
from skimage.feature import hog # Nền tảng AI, máy học: đưa sao học sau ra vậy, chưa học chưa ra được. Thuật toán: HOG, SVC (Machine learling), trích xuất đặt trưng ra một vector
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score # accuracy_score : Độ chính xác, độ tin cậy
from keras.datasets import mnist # Deep learning, huấn luyện mô hình, train module, thư viện mnist
#from google.colab.patches import cv2_imshowpy

Tiếp theo ta sẽ tính hog. Ta dùng orientations=9,pixels_per_cell=(14,14),cells_per_block=(1,1).

In [ ]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()

X_train_feature = []
for i in range(len(X_train)):
    feature = hog(X_train[i],orientations=9,pixels_per_cell=(14,14),cells_per_block=(1,1),block_norm="L2")
    X_train_feature.append(feature)
X_train_feature = np.array(X_train_feature,dtype = np.float32)

#cho x_test
X_test_feature = []
for i in range(len(X_test)):
    feature = hog(X_test[i],orientations=9,pixels_per_cell=(14,14),cells_per_block=(1,1),block_norm="L2")
    X_test_feature.append(feature)
X_test_feature = np.array(X_test_feature,dtype=np.float32)

Tiếp theo ta build model vào predict

predict Tính toán độ chính xác mô hình

In [ ]:
model = LinearSVC(C=10)
model.fit(X_train_feature,y_train)
y_pre = model.predict(X_test_feature)
print(accuracy_score (y_test,y_pre)) # In ra độ chính xác mô hình

Predict trên ảnh

In [ ]:
image = cv2.imread("/content/drive/My Drive/TriTueNhan TAo/digit.jpg")
im_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
im_blur = cv2.GaussianBlur (im_gray, (5,5),0)
contours, hierachy = cv2.findContours(thre, c2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # Đi tìm ra những nét của chữ số, canny, edge: phát hiện cạnh biên
rects = [cv2.boundingRect(cnt) for cnt in contours] #
im, thre= cv2.threshold (im_blur,90,255,cv2.THRESH_BINARY_INV)

Sau đó predict digit của mỗi box.

In [ ]:
for i in contours:
    (x,y,w,h) = cv2.boundingRect(i)
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),3)
    roi = thre[y:y+h,x:x+w]
    roi = np.pad(roi,(20,20),'constant',constant_values=(0,0))
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
    roi = cv2.dilate(roi, (3, 3))

    # Calculate the HOG features - Tính toán độ chính xác mô hình
    roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1),block_norm="L2")
    nbr = model.predict(np.array([roi_hog_fd], np.float32)) # np.array([roi_hog_fd], np.float32) => Hình đang xử lý
    cv2.putText(image, str(int(nbr[0])), (x, y), cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 255), 3) # putText : Đèn chữ lên tấm hình, putText(Ảnh vẽ đè lên, str(int(nbr[0])) - đè số 9 lên ảnh, (x, y) - tọa độ vẽ lên, FONT_HERSHEY_DUPLEX - kiểu phong, 2 - độ dày, (0, 255, 255) - màu vàng)
    cv2_imshow(image)
# cv2.imwrite("image_pand.jpg",image)
cv2.waitKey()
cv2.destroyAllWindows()

# Bài tập nâng cao
Anh chị hãy in ra độ chính xác của mô hình vừa huấn luyện?
Anh chị chỉ hiển thị những chữ số có độ chính xác trên 98%?
- Tìm ra độ chính xác từng số
- bỏ vào hàm if